In [7]:
import requests
import csv
import time

In [1]:
main_overpass_api = "https://overpass-api.de/api/interpreter"
lz4_overpass_api='https://lz4.overpass-api.de/api/interpreter'
osm_overpass_api='https://overpass.openstreetmap.ru/api/interpreter'

BUNDES_GROUPS = [
    ["Baden-Württemberg"],
    ["Bayern"],
    ["Berlin", "Bremen"],
    ["Brandenburg","Hamburg"],
    ["Hessen","Mecklenburg-Vorpommern","Rheinland-Pfalz"],
    ["Niedersachsen"],
    ["Nordrhein-Westfalen"],
    ["Saarland","Sachsen","Sachsen-Anhalt", "Schleswig-Holstein","Thüringen"]
]

Große Bundeslaeder = Bayern, (Berlin, Brandenburg), Hamburg, Nordrhein-Westfalen. Einzelne fragen
Mitlere Bundeslaeder= Baden-Württemberg, Bremen, Niedersacchsen 
Kleine = Hessen, Mecklenburg-Vorpommern, Rheinland-Pfalz, Saarland, Sachsen, Sachsen-Anhalt, Schleswig-Holstein, Thüringen
mini = 

In [ ]:
all_results=[]
for groups in BUNDES_GROUPS:
    for region in groups:
        query = f"""[out:json][timeout:180];
                area["name"="{region}"]->.searchArea;
            (
                node["name"~"(AWO|Arbeiterwohlfahrt)",i](area.searchArea);
                way["name"~"(AWO|Arbeiterwohlfahrt)",i](area.searchArea);
                relation["name"~"(AWO|Arbeiterwohlfahrt)",i](area.searchArea);

                node(area.searchArea)["operator"~"(AWO|Arbeiterwohlfahrt)",i];
                way(area.searchArea)["operator"~"(AWO|Arbeiterwohlfahrt)",i];
                relation(area.searchArea)["operator"~"(AWO|Arbeiterwohlfahrt)",i];

                node(area.searchArea)["brand"~"(AWO|Arbeiterwohlfahrt)",i];
                way(area.searchArea)["brand"~"(AWO|Arbeiterwohlfahrt)",i];
                relation(area.searchArea)["brand"~"(AWO|Arbeiterwohlfahrt)",i];

            );
            out center;
        """
        response=requests.get(main_overpass_api, params={'data':query})
       
        try:
            result = response.json()
        except Exception as e:
            print(f"Error for {region}, {e}")
            continue
   # all_results.append(result)
    #extracted = extract_elements(result, el)
        for el in result.get("elements", []):
            tags=el.get("tags", {})
            all_results.append({
                "osm_id": el.get("id"),
                "region": region,
                "type": el.get("type"),
                "name": tags.get("name", ""),
                "street": tags.get("addr:street", ""),
                "housenumber": tags.get("addr:housenumber", ""),
                "postcode": tags.get("addr:postcode", ""),
                "city": tags.get("addr:city", ""),
                "lat": el.get("lat") or el.get("center", {}).get("lat"),
                "lon": el.get("lon") or el.get("center", {}).get("lon"),
                "phone": tags.get("contact:phone", tags.get("phone", "")),
                "email": tags.get("contact:email", tags.get("email", "")),
                "website": tags.get("contact:website", tags.get("website", "")),
                "amenity": tags.get("amenity", "")
            })
        time.sleep(10)
df=pd.DataFrame(all_results)
df.to_csv(f"New_csv_all.csv", index=False, encoding='utf-8')
print(f'Saved {len(df)} results total')
#print('For following Districts {} Saved {} results'.format(BUNDESLAENDER, len(df)))

AttributeError: 'DataFrame' object has no attribute 'pd'

In [185]:
len(all_results)

678

In [203]:
df.region.value_counts()

region
Nordrhein-Westfalen       1236
Bayern                     678
Niedersachsen              510
Thüringen                  300
Baden-Württemberg          299
Sachsen                    280
Hessen                     237
Schleswig-Holstein         190
Brandenburg                163
Mecklenburg-Vorpommern     156
Sachsen-Anhalt             142
Saarland                   121
Rheinland-Pfalz             94
Berlin                      84
Bremen                      69
Hamburg                     40
Name: count, dtype: int64

In [198]:
pattern_upper = r"\bAWO\b"
has_word = df['name'].str.contains(pattern_word, case=False, na=False)
#has_any = big['name'].str.contains("awo", case=False, na=False)
mask = has_word|(~has_any)
df_filtered = df.loc[has_word].reset_index(drop=True)

In [202]:
df_wrong_findings = df.loc[~has_word].reset_index(drop=True)
df_wrong_findings.sample(10)


,osm_id,region,type,name,street,housenumber,postcode,city,lat,lon,phone,email,website,amenity
1722,754925178,Nordrhein-Westfalen,way,"Kindertageseinrichtung ""Traumland""",Josefstraße,35,52222.0,Stolberg,50.780629,6.246176,NaN,NaN,https://www.awo-kisa.de/kita-traumland/,kindergarten
1010,8950746874,Niedersachsen,node,Begegnungszentrum Mittelpunkt Hüttenstraße,Gubener Straße,1,31655.0,Stadthagen,52.327717,9.180325,+49 5721 98 09 573,NaN,https://awo-kv-schaumburg.de/angebote/begegnun...,NaN
192,2567941217,Bayern,node,NaN,NaN,NaN,NaN,NaN,49.410006,11.047433,NaN,NaN,NaN,recycling
778,1026972483,Hessen,way,Altenzentrum Wabern,Kurfürstenstraße,12,34590.0,Wabern,51.103743,9.345243,NaN,NaN,NaN,social_facility
294,35004514,Bayern,way,Kindertagesstätte an der Hohenbrunner Straße,Hohenbrunner Straße,34,85579.0,Neubiberg,48.073576,11.670691,NaN,NaN,NaN,kindergarten
1032,12409617318,Niedersachsen,node,NaN,Bahnhofstraße,NaN,37574.0,Einbeck,51.797584,9.922742,NaN,NaN,NaN,bicycle_parking
607,150075968,Brandenburg,way,Wohnstätte für Senioren,Clara-Zetkin-Straße,14,3046.0,Cottbus,51.758934,14.309663,NaN,NaN,NaN,social_facility
743,151868648,Hessen,way,Käthe-Richter-Haus,Eberhard-Wildermuth-Straße,15,34121.0,Kassel,51.303568,9.480854,NaN,NaN,NaN,social_facility
1780,958215322,Nordrhein-Westfalen,way,NaN,NaN,NaN,NaN,NaN,50.926318,8.441696,NaN,NaN,NaN,parking
2353,159511738,Thüringen,way,NaN,Karl-Marx-Straße,28,4617.0,Rositz,51.016030,12.372561,NaN,NaN,NaN,nursing_home


In [201]:
df_filtered.shape

(2138, 14)

In [4]:
from osm_script import osm_extractor_groups, fetch_osm_region

In [5]:
df=osm_extractor_groups(BUNDES_GROUPS)

Error for Hamburg, Expecting value: line 1 column 1 (char 0)
Error for Niedersachsen, Expecting value: line 1 column 1 (char 0)


In [7]:
df.region.value_counts()

region
Nordrhein-Westfalen       1236
Bayern                     678
Thüringen                  300
Baden-Württemberg          299
Sachsen                    280
Hessen                     237
Schleswig-Holstein         190
Brandenburg                163
Mecklenburg-Vorpommern     156
Sachsen-Anhalt             142
Saarland                   121
Rheinland-Pfalz             94
Berlin                      84
Bremen                      69
Name: count, dtype: int64

In [8]:
ham = fetch_osm_region("Hamburg")

In [9]:
len(ham)

40

In [10]:
nieder = fetch_osm_region("Niedersachsen")

In [11]:
len(nieder)

510

In [141]:
big_filtered.to_csv("total_results.csv")

for wrong findings, there was one false negative, AWOlino Pentzberg. As next step I could try to search for AWO in county regions ? Bezirksverband or give Bezirksverband name and then ortsvereine? But first crosscheck which if urls in url sheet are same as urls in osm-sheet

In [145]:
df = pd.read_csv("total_results.csv")
domains=pd.read_excel("AWO_domains_with_impressum_data.xlsx", sheet_name="impressum_extracted")

domains.sample(5)


,id,domain,name,strasse,hausnummer,plz,ort,email
903,946,bildungswerk-rheinland.de,AWO Dienstleistungs- und Service GmbH,Dreikaiserweg,4,56068.0,Koblenz,bildungswerk@awo-rheinland.de
56,99,awo-kita-neuburg.de,Arbeiterwohlfahrt Bezirksverband Schwaben e.V.,Sonnenstraße,10,86391.0,Stadtbergen,NaN
252,295,awo-seniorenzentren.awo-ww.de,Arbeiterwohlfahrt Bezirksverband Westliches We...,Kronenstraße,63 - 69,44139.0,Dortmund,NaN
15,58,awo-pflegedienst.de,AWO Jugend- und Sozialdienste gGmbH,Wilhelmstraße,7,30171.0,Hannover,info@awo-pflegedienst.de
1340,1383,awo-rhk.de,Arbeiterwohlfahrt Kreisverband Rhein-Hunsrück ...,Winterbachstr.,4,55487.0,Sohren,info@awo-rhk.de


In [156]:
#website column is wrong
df['website'] = df['website'].str.replace(r'https://', '', regex=True)

In [157]:
df.sample(5)

,Unnamed: 0,osm_id,type,name,street,housenumber,postcode,city,lat,lon,phone,email,website,amenity
1703,1703,258163570,way,AWO-Seniorenheim,Friedrich-Bauer-Weg,6,86690.0,Mertingen,48.655290,10.800146,NaN,NaN,NaN,social_facility
1060,1060,166224032,way,"AWO Integrative Kindertageseinrichtung ""Bollen...",Bollenberger Busch,29,42781.0,Haan,51.199320,7.022139,NaN,NaN,NaN,childcare
819,819,2463762745,node,AWO Altenwohnungen,Badeweg,50,32584.0,Löhne,52.218649,8.741948,NaN,NaN,NaN,NaN
1625,1625,48514930,way,AWO Kindergarten Pusteblume,Cerveteristraße,12,NaN,NaN,48.167952,11.225971,NaN,NaN,awo-obb.de/kinder,kindergarten
1222,1222,771933411,way,AWO Kinder- und Jugendzeltplatz,NaN,NaN,NaN,NaN,51.501154,6.439710,NaN,NaN,NaN,NaN


In [158]:
domains1=domains.assign(InOsm=domains.domain.isin(df.website).astype(int))

In [150]:
domains1.columns

Index(['id', 'domain', 'name', 'strasse', 'hausnummer', 'plz', 'ort', 'email',
       'InOsm'],
      dtype='object')

In [159]:
domains1.InOsm.value_counts()

InOsm
0    1661
1       5
Name: count, dtype: int64

In [160]:
df1 = df.assign(in_domains=df.website.isin(domains1.domain).astype(int))

In [161]:
df1.in_domains.value_counts()

in_domains
0    1826
1       9
Name: count, dtype: int64